In [1]:
# Imports
import numpy as np
import pandas as pd

from pylab import rcParams
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# Customise our plotting settings
rcParams['figure.figsize'] = 10, 5
sns.set_style('whitegrid')

import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report

import nltk

In [2]:

from matplotlib.colors import ListedColormap

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV

In [4]:
data = pd.read_csv('train.csv')
sample_data = pd.read_csv('sample_submission.csv')
testing = pd.read_csv('test.csv')

In [5]:
train = data.copy()
sample = sample_data.copy()
test = testing.copy()

In [6]:
# Make lower case
train['message'] = train['message'].str.lower()
test['message'] = test['message'].str.lower()

In [7]:
# Remove urls
pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
subs_url = r'url-web'
train['message'] = train['message'].replace(to_replace = pattern_url, value = subs_url, regex = True)
test['message'] = test['message'].replace(to_replace = pattern_url, value = subs_url, regex = True)

In [8]:
#Remove email
import re
train['message'] = train['message'].apply(lambda tweet: re.sub('[a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+', '', tweet))
test['message'] = test['message'].apply(lambda tweet: re.sub('[a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+', '', tweet))
train.head()

,sentiment,message,tweetid
0,1,polyscimajor epa chief doesn't think carbon di...,625221
1,1,it's not like we lack evidence of anthropogeni...,126103
2,2,rt @rawstory: researchers say we have three ye...,698562
3,1,#todayinmaker# wired : 2016 was a pivotal year...,573736
4,1,"rt @soynoviodetodas: it's 2016, and a racist, ...",466954


In [9]:
#Remove rt and mentions
train['message'] = train['message'].apply(lambda tweet: re.sub('rt', '', tweet))
test['message'] = test['message'].apply(lambda tweet: re.sub('rt', '', tweet))
train.head()

,sentiment,message,tweetid
0,1,polyscimajor epa chief doesn't think carbon di...,625221
1,1,it's not like we lack evidence of anthropogeni...,126103
2,2,@rawstory: researchers say we have three year...,698562
3,1,#todayinmaker# wired : 2016 was a pivotal year...,573736
4,1,"@soynoviodetodas: it's 2016, and a racist, se...",466954


In [10]:
# Strip out punctuation marks and numerals
import string
def remove_punctuation_numbers(post):
    punc_numbers = string.punctuation + '0123456789'
    return ''.join([l for l in post if l not in punc_numbers])

train['message'] = train['message'].apply(remove_punctuation_numbers)
test['message'] = test['message'].apply(remove_punctuation_numbers)
train.head()

,sentiment,message,tweetid
0,1,polyscimajor epa chief doesnt think carbon dio...,625221
1,1,its not like we lack evidence of anthropogenic...,126103
2,2,rawstory researchers say we have three years ...,698562
3,1,todayinmaker wired was a pivotal year in the...,573736
4,1,soynoviodetodas its and a racist sexist clim...,466954


In [11]:
#Remove multiple spaces
train['message'] = train['message'].apply(lambda tweet: ' '.join(tweet.split()))
test['message'] = test['message'].apply(lambda tweet: ' '.join(tweet.split()))

In [12]:
#Tokenisation
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
tokeniser = TreebankWordTokenizer()
train['message'] = train['message'].apply(tokeniser.tokenize)
test['message'] = test['message'].apply(tokeniser.tokenize)

In [13]:
#lemmatization
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
def mbti_lemma(words, lemmatizer):
    return [lemmatizer.lemmatize(word) for word in words]

train['message'] = train['message'].apply(mbti_lemma, args=(lemmatizer, ))
test['message'] = test['message'].apply(mbti_lemma, args=(lemmatizer, ))

[nltk_data] Downloading package wordnet to C:\Users\Lenovo
[nltk_data]     X230\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
# Removing Stop Words
from nltk.corpus import stopwords
print(stopwords.words('english'))
def remove_stop_words(tokens):
    return [t for t in tokens if t not in stopwords.words('english')]
train['message'] = train['message'].apply(remove_stop_words)
test['message'] = test['message'].apply(remove_stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# To fit models on unbalanced data skip the balancing section

# Balancing data

In [ ]:
train_sum = train[['sentiment', 'message']].groupby('sentiment').count()

In [ ]:
# plot by sentimental classes
train_sum.sort_values('message', ascending=False).plot(kind='bar')
plt.show()

In [ ]:
# As usual, we start by importing our modules
from sklearn.utils import resample

In [ ]:
#class size equals to 50% of the majority class(can change it to any number which will give best result)



class_size = int(len(train[train['sentiment']==1]))# Upsampling

#class_size = int(len(train[train['sentiment']==1])/2) # upsample minority class + downsample majority class

#class_size = int(len(train[train['sentiment']==-1])) # DownSampling

#class_size = 6000

class_size

#uncomment the others to try them out

In [ ]:
one = train[train['sentiment']==1]
two = train[train['sentiment']==2]

In [ ]:

one_downsampled = resample(one,
                          replace=False, # sample without replacement (no need to duplicate observations)
                          n_samples=class_size, # match number in minority class
                          random_state=27) # reproducible results

# Combine downsampled majority class with minority class
downsampled = pd.concat([one_downsampled, two])

# Check new class counts
downsampled['sentiment'].value_counts()

In [ ]:
# Upsample minority
two_upsampled = resample(two,
                          replace=True, # sample with replacement (we need to duplicate observations)
                          n_samples=class_size, # match number in minority class
                          random_state=27) # reproducible results

# Combine upsampled minority class with majority class
upsampled = pd.concat([two_upsampled, one_downsampled])

# Check new class counts
upsampled['sentiment'].value_counts()

In [ ]:
zero = train[train['sentiment']==0]

In [ ]:
zero_upsampled = resample(zero,
                          replace=True, # sample with replacement (we need to duplicate observations)
                          n_samples=class_size, # match number in minority class
                          random_state=27) # reproducible results

# Combine upsampled minority class with majority class
upsampled0 = pd.concat([zero_upsampled, upsampled])

# Check new class counts
upsampled0['sentiment'].value_counts()

In [ ]:
minus_one = train[train['sentiment']==-1]

In [ ]:
minus_one_upsampled = resample(minus_one,
                          replace=True, # sample with replacement (we need to duplicate observations)
                          n_samples=class_size, # match number in minority class
                          random_state=27) # reproducible results

# Combine upsampled minority class with majority class
upsampled_ = pd.concat([minus_one_upsampled, upsampled0])

# Check new class counts
upsampled_['sentiment'].value_counts()

In [ ]:
train=upsampled_ 
train

In [ ]:
train_sum = train[['sentiment', 'message']].groupby('sentiment').count()

In [ ]:
# Plot by sentimental classes
train_sum.sort_values('message', ascending=False).plot(kind='bar')
plt.show()

# Fitting models on data

In [15]:
vect = CountVectorizer(stop_words='english', min_df= .01)
A = train['message'].astype(str)
B = vect.fit_transform(A)

In [16]:
from sklearn.model_selection import train_test_split

X = B
y = train['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [ ]:
names = ['Logistic Regression',
         'Linear SVC',
         'Naive Bayes',
         'Decision Tree', 'Random Forest',
         'One VS the rest', 'KNN', 'AdaBoost']

classifiers = [
    LogisticRegression(multi_class='ovr'),
    SVC(),
    MultinomialNB(),
    DecisionTreeClassifier(max_depth=5),
    KNeighborsClassifier(),
    RandomForestClassifier(),
    OneVsRestClassifier(LinearSVC(), n_jobs=-1),
    AdaBoostClassifier()
]

results = []
models = {}
confusion = {}
class_report = {}


for name, clf in zip(names, classifiers):
    print ('Fitting {:s} model...'.format(name))
    run_time = %timeit -q -o clf.fit(X_train, y_train)

    print ('... predicting')
    y_pred = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)

    print ('... scoring')
    accuracy  = metrics.accuracy_score(y_train, y_pred)
    precision = metrics.precision_score(y_train, y_pred,average='weighted')
    recall    = metrics.recall_score(y_train, y_pred, average='weighted')

    f1        = metrics.f1_score(y_train, y_pred,average='weighted')
    f1_test   = metrics.f1_score(y_test, y_pred_test,average='weighted')

    # Save the results to dictionaries
    models[name] = clf
    confusion[name] = metrics.confusion_matrix(y_train, y_pred)
    class_report[name] = metrics.classification_report(y_train, y_pred)

    results.append([name, accuracy, precision, recall, f1, f1_test, run_time.best])


results = pd.DataFrame(results, columns=['Classifier', 'Accuracy', 'Precision', 'Recall', 'F1 Train', 'F1 Test', 'Train Time'])
results.set_index('Classifier', inplace= True)

Fitting Logistic Regression model...


C:\Users\Lenovo X230\Anaconda3\ana\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


... predicting
... scoring
Fitting Linear SVC model...


C:\Users\Lenovo X230\Anaconda3\ana\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Lenovo X230\Anaconda3\ana\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Lenovo X230\Anaconda3\ana\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Lenovo X230\Anaconda3\ana\lib\site-packa

In [ ]:
results.sort_values('F1 Train', ascending=False)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
results.sort_values('F1 Train', ascending=False, inplace=True)
results.plot(y=['F1 Test'], kind='bar', ax=ax[0], xlim=[0,1.1], ylim=[0.30,0.92])
results.plot(y='Train Time', kind='bar', ax=ax[1])

# Cross Validation on all models

In [ ]:
#Choose model name to do crossvalidation on specific model
model = models['RBF SVM']
print(cross_val_score(model, X, y))

In [ ]:
# Cross validation for all models
cv = []
for name, model in models.items():
    print ()
    print(name)
    scores = cross_val_score(model, X, y, cv=10)
    print("Accuracy: {:0.2f} (+/- {:0.4f})".format(scores.mean(), scores.std()))
    cv.append([name, scores.mean(), scores.std() ])

cv = pd.DataFrame(cv, columns=['Model', 'CV_Mean', 'CV_Std_Dev'])
cv.set_index('Model', inplace=True)

# using pipeline and TfidfVectorizer to fit models

In [ ]:
from sklearn.model_selection import train_test_split

X = train['message'].astype(str)
y = train['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
names = ['Logistic Regression',
         'Linear SVC','Naive Bayes',
         'Decision Tree', 'Random Forest','One VS the rest'  'AdaBoost']

In [ ]:
classifiers = [Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf',LogisticRegression(multi_class='ovr')),]),
                      
    Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf',SVC(kernel="linear")),]),
                      
    Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', MultinomialNB()),]),
                      
    Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf',DecisionTreeClassifier(max_depth=5)),]),
                      
    Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf',RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)),]),
               
    Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf',OneVsRestClassifier(LinearSVC(), n_jobs=-1)),])
                      
    Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf',AdaBoostClassifier()),]) ]

In [ ]:

results = []
models = {}
confusion = {}
class_report = {}


for name, clf in zip(names, classifiers):
    print ('Fitting {:s} model...'.format(name))
    run_time = %timeit -q -o clf.fit(X_train, y_train)

    print ('... predicting')
    y_pred = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)

    print ('... scoring')
    accuracy  = metrics.accuracy_score(y_train, y_pred)
    precision = metrics.precision_score(y_train, y_pred,average='weighted')
    recall    = metrics.recall_score(y_train, y_pred, average='weighted')

    f1        = metrics.f1_score(y_train, y_pred,average='weighted')
    f1_test   = metrics.f1_score(y_test, y_pred_test,average='weighted')

    # Save the results to dictionaries
    models[name] = clf
    confusion[name] = metrics.confusion_matrix(y_train, y_pred)
    class_report[name] = metrics.classification_report(y_train, y_pred)

    results.append([name, accuracy, precision, recall, f1, f1_test, run_time.best])


results = pd.DataFrame(results, columns=['Classifier', 'Accuracy', 'Precision', 'Recall', 'F1 Train', 'F1 Test', 'Train Time'])
results.set_index('Classifier', inplace= True)

In [ ]:
results.sort_values('F1 Train', ascending=False)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
results.sort_values('F1 Train', ascending=False, inplace=True)
results.plot(y=['F1 Test'], kind='bar', ax=ax[0], xlim=[0,1.1], ylim=[0.30,0.92])
results.plot(y='Train Time', kind='bar', ax=ax[1])

# Cross Validation for all the models

In [ ]:
#Choose model name to do crossvalidation on specific model
model = models['Logistic Regression']
print(cross_val_score(model, X, y))

In [ ]:
#Cross validation for all models
cv = []
for name, model in models.items():
    print ()
    print(name)
    scores = cross_val_score(model, X, y, cv=10)
    print("Accuracy: {:0.2f} (+/- {:0.4f})".format(scores.mean(), scores.std()))
    cv.append([name, scores.mean(), scores.std() ])

cv = pd.DataFrame(cv, columns=['Model', 'CV_Mean', 'CV_Std_Dev'])
cv.set_index('Model', inplace=True)

# Grid Search On Best Perfoming Models

In [ ]:
# Grid search and crossvalidation on SVC
nfolds = 2

Cs = [0.001, 0.01, 0.1, 1, 0,25, 10]
gammas = [0.001, 0.01, 0.1, 1,2]

param_grid = {
    'C'     : Cs,
    'gamma' : gammas
    }

grid_SVM = GridSearchCV(SVC(), param_grid, scoring='f1', cv=nfolds)
grid_SVM.fit(X_train, y_train)